# Chains

Hello everyone! Here, we will explore the most important concept in langchain, "chaining" 

Concepts covered in this notebook are:
1. [Basic Chaining](#1)
2. [Chaining under the Hood](#2)
3. [Create Custom Chain Step](#3)
4. [Parallel Chains](#4)
5. [Branched Chains](#5)

Prerequisites (I hope you have understood these concepts):
1. language model
2. prompt template
3. output parser

## Load environment variables

In [2]:
from dotenv import load_dotenv

load_dotenv()

## By default, load_dotenv() will assign environment variables into os.environ, like following code:
## See environment variables in .env file
# import os
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] =  os.getenv('LANGCHAIN_API_KEY')
# os.environ["OPENAPI_KEY"] =  os.getenv('OPENAI_KEY')

True

Let's create language model instance, prompt template, and output parser

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser 

model = ChatOpenAI(model="gpt-3.5-turbo")

prompt_template = ChatPromptTemplate([
    ("system", "Translete the following into {language}"),
    ("user", "{text}")
])

parser = StrOutputParser()

# Basic Chaining

"Chaining" is operation to combine sequential processes — for example from prompt templates to output parsers— using the pipe (`|`) operator.

In [4]:
chain = prompt_template | model | parser

chain.invoke({"language": "indonesia", "text": "How are you?"})

'Apa kabar?'

## 2 - Chaining Under the Hood <div id="2"></div>

For more details, refer to the file `0_chain_under_the_hood.ipynb`.

Chaining is a feature of the Langchain Expression Language (LCEL) that provides an intuitive and straightforward way to link Langchain components. To fully understand chaining under the hood, it's essential to grasp some underlying concepts.

### 2.1 - `Runnable` Interface

The `Runnable` interface represents a unit of work that can be invoked, batched, streamed, transformed, and composed. Many Langchain components, such as chat models, prompt templates, and output parsers, are built using the `Runnable` interface. This interface essentially includes two key methods:

- `.invoke()`: This method executes the runnable and returns the result.
- `.__or__()`: This method overrides the pipe operator (`|`) in Python.

Let's examine an example to demonstrate that the `model`, `prompt_template`, and `parser` we have are implemented using the `Runnable` interface.

All three components have an `.invoke()` method. Let's call this method on each of them.

In [5]:

out = prompt_template.invoke({"language": "indonesia", "text": "How are you?"})
out = model.invoke(out)
out = parser.invoke(out)

out

'Apa kabar?'

All three components also have a `__or__()` method. Note that the `__or__()` method is for operator overloading, meaning you can use the `|` pipe operator with the same result as calling the `__or__()` method.

In [6]:
out1 = prompt_template.__or__(model).__or__(parser)

out2 = prompt_template | model | parser

print("Out1", out1)
print("Out2", out2)

print("Class of Out1: ", out1.__class__.__name__)
print("Class of Out2: ", out2.__class__.__name__)

Out1 first=ChatPromptTemplate(input_variables=['language', 'text'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='Translete the following into {language}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))]) middle=[ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000181AE775460>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000181AE776A50>, openai_api_key=SecretStr('**********'), openai_proxy='')] last=StrOutputParser()
Out2 first=ChatPromptTemplate(input_variables=['language', 'text'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='Translete the following into {language}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))]) middle=[ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000181AE775460

What is the output of the `__or__()` method? Based on the code above, it is the `RunnableSequence` class. What is that?

## 2.2 - `RunnableSequence`

`RunnableSequence` is a class that allows chaining multiple `Runnable`s together. As previously mentioned, chaining `Runnable`s using the pipe operator produces a `RunnableSequence`. However, a formal way to create a `RunnableSequence` is by using the `RunnableSequence` class directly.

The `RunnableSequence` class accepts runnable components as arguments, in the specified order.

In [7]:
from langchain.schema.runnable import RunnableSequence

# Create runnable sequence
runnable_seq = RunnableSequence(prompt_template, model, parser)

# alternatives. Note that middle must be a list of runnable components
# chain = RunnableSequence(first=prompt_template, middle=[model], last=parser)

Important property of `RunnableSequence` is `steps`:

- `steps`: A list of `Runnable` objects. Let's print it out.

In [8]:

runnable_seq.steps

[ChatPromptTemplate(input_variables=['language', 'text'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='Translete the following into {language}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))]),
 ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000181AE775460>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000181AE776A50>, openai_api_key=SecretStr('**********'), openai_proxy=''),
 StrOutputParser()]

Important methods in `RunnableSequence` are:

- `__or__()`: This method overloads the pipe operator, producing another `RunnableSequence` that chains the current `RunnableSequence` with the next component.
- `.invoke()`: This method calls the `.invoke()` method of each `Runnable` object in sequence, passing the output of one as the input to the next.

Let's examine this with code.

In [9]:
basic_seq = RunnableSequence(prompt_template, model)
advance_seq = basic_seq.__or__(parser) # equivalent to `advance_seq = basic_seq | parser`.

print("Basic Seq class name:", basic_seq.__class__.__name__)
print("Advance Seq class name:", advance_seq.__class__.__name__)

Basic Seq class name: RunnableSequence
Advance Seq class name: RunnableSequence


In [10]:
runnable_seq.invoke(({"language": "indonesia", "text": "How are you?"}))

'Apa kabar?'

### 2.3 - Summary Chaining Under The Hood

Chaining basically is creating `RunnableSequence` by chaining `Runnable` components using operator pipe (`|`).

## 3 - Chain Custom Chain Step

We have used several chain step components. All of that categorized as `Runnable` including:
- Chat Models
- LLM
- Prompt Templates
- Output Parsers
- Retrievers (next tutorial)
- Tools (next tutorial)

Additionally, we can create our own `Runnable` component by wrapping a function with the `RunnableLambda` class.

In [11]:
from langchain.schema.runnable import RunnableLambda

multiply = RunnableLambda(lambda x: x[0]*x[1]) # lamdba function only accept one argument
square = RunnableLambda(lambda x: x**2)
tenth = RunnableLambda(lambda x : x / 10)

We can now chain and invoke these runnable function

In [12]:
math_chain = multiply | square | tenth

math_chain.invoke((5, 2))

10.0

Let's chain common langchain components with custom runnable function

In [13]:
# Define additional processing steps using RunnableLambda
uppercase_output = RunnableLambda(lambda x: x.upper())
count_words = RunnableLambda(lambda x: f"Word count: {len(x.split())}")

# Create the combined chain using LangChain Expression Language (LCEL)
chain = prompt_template | model | parser | uppercase_output | count_words

chain.invoke({"language": "indonesia", "text": "How are you?"})

'Word count: 2'

## 4 - Pararel Chains

We can create Pararel chain using class `RunnablePararel` or plain dictionary `Dict(string, Runnable)`

In [25]:
square = RunnableLambda(lambda x: x**2)
tenth = RunnableLambda(lambda x : x / 10)

def combine(square, tenth):
    return {"square": square, "tenth": tenth}

# using dictionary to create pararel chaining
chain = {"square": square, "tenth": tenth} | RunnableLambda(lambda x: combine(x["square"], x["tenth"]))

chain.invoke(10)

{'square': 100, 'tenth': 1.0}

In [26]:
from langchain.schema.runnable import RunnableParallel
# Create the parallel runnable
parallel_chain = RunnableParallel(branches={"square": square, "tenth": tenth})

# Chain the parallel runnable with the combining function
chain = parallel_chain | RunnableLambda(lambda x: combine(x["branches"]["square"], x["branches"]["tenth"]))

chain.invoke(10)

{'square': 100, 'tenth': 1.0}

## 5 - Branched Chains

A `RunnableBranch` is a special type of runnable that allows you to define a set of conditions and runnables to execute based on the input. 

In [6]:
from langchain.schema.runnable import RunnableBranch, RunnableLambda

square = RunnableLambda(lambda x: x**2)
tenth = RunnableLambda(lambda x : x / 10)

chain = RunnableBranch(
    (lambda x: x%2 == 0, RunnableLambda(lambda x: f"{x} is even")),
     RunnableLambda(lambda x: f"{x} is odd")
)

chain.invoke(3)

'3 is odd'